In [ ]:
!pip install ../input/pathfinder/python_box-5.4.1-py3-none-any.whl
!pip install ../input/timm0412/timm-0.4.12-py3-none-any.whl

In [ ]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
from box import Box
from timm import create_model
import cv2

import torch
import torch.nn as nn
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
import albumentations as A
from albumentations.pytorch import ToTensorV2

warnings.filterwarnings("ignore")

In [ ]:
# sys.path.append("../input/timmswin")

In [ ]:
config = {'root': '/kaggle/input/petfinder-pawpularity-score/',
          'image_size': 224,
          'dataloader':{
              'batch_size': 64,
              'shuffle': False,
              'num_workers': 4,
              'pin_memory': True,
              'drop_last': False,
          },
          'model':{
              'name': 'swin_large_patch4_window7_224_in22k',
              'freeze_backbone': False,
              'output_dim': 1
          },
          'trainer':{
              'gpus': 1
          },
          'test':{
              'n_fold': 5,
              'trn_folds': [0, 1, 2, 3, 4],
              'tta_steps': 10
          }
}

config = Box(config)

In [ ]:
test_aug = A.Compose(
    [
        A.Resize(config.image_size, config.image_size, p=1),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
        ToTensorV2(),
    ],
    p=1.0,
)

In [ ]:
class PetfinderDataset(Dataset):
    def __init__(self, df, augmentations):
        self._X = df["Id"].values
        self._augmentations = augmentations

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self._augmentations(image=image)["image"]
        return image

In [ ]:
class Model(pl.LightningModule):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.__build_model()
        self.save_hyperparameters(cfg)

    def __build_model(self):
        self.backbone = create_model(
            config.model.name, pretrained=False, num_classes=0, in_chans=3
        )
        num_features = self.backbone.num_features
        self.fc = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(num_features, 768),
            nn.Linear(768, 192),
            nn.Linear(192, 64),
            nn.Linear(64, config.model.output_dim)
        )
        
    def forward(self, x):
        f = self.backbone(x)
        out = self.fc(f)
        return out

    def predict_step(self, batch, batch_idx):
        x = batch
        out = self(x)
        return out.sigmoid()

In [ ]:
def get_weights_path(fold):
    path = "../input/swin-t-large"
    return os.path.join(path, f"{config.model.name}/best_loss_v{fold}.ckpt")

In [ ]:
df = pd.read_csv(os.path.join(config.root, "test.csv"))
df["Id"] = df["Id"].apply(lambda x: os.path.join(config.root, "test", x + ".jpg"))

In [ ]:
def get_fold_predictions(fold):
    weight = get_weights_path(fold)
    model = Model.load_from_checkpoint(checkpoint_path=weight, cfg=config)
    trainer = pl.Trainer(**config.trainer)
    tta_preds = []
    for tta in range(config.test.tta_steps):
        ds = PetfinderDataset(df)
        dl = DataLoader(ds, **config.dataloader)
        predictions = trainer.predict(model, dl)
        tta_preds.append(torch.cat([x * 100 for x in predictions]).detach().cpu().numpy())
        
    predictions = np.mean(np.column_stack(tta_preds), axis=1)
    return predictions

In [ ]:
fold_predictions = []
for fold in config.test.trn_folds:
    preds = get_fold_predictions(fold)
    fold_predictions.append(preds)
    
final_predictions = np.mean(np.column_stack(fold_predictions), axis=1)

In [ ]:
submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
submission["Pawpularity"] = final_predictions
submission.to_csv("submission.csv", index=False)
submission.head()